In [12]:
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

load_dotenv()

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

examples = [
    {"title": "Parasite", "answer": "🏠👨‍👩‍👧‍👦🪳"},
    {"title": "Titanic", "answer": "🚢❤️🧊"},
    {"title": "Iron Man", "answer": "🦸‍♂️🤖💥"},
    {"title": "The Lord of the Rings: The Fellowship of the Ring", "answer": "💍🧙‍♂️🗻"},
    {"title": "Mamma Mia!", "answer": "🏝️💃🎵"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {title}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "You are a film critic with an exceptional talent for expressing movies using only three emojis. Your expertise lies in distilling the essence, themes, and key elements of films into concise emoji representations."),
        MessagesPlaceholder(variable_name="history"),
        example_prompt,
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content}
    )
    print(result)


In [13]:
invoke_chain("What do you know about Star Wars: Episode IV?")

content='🌌⚔️🤖'


In [14]:
invoke_chain("What do you know about A Fistful of Dollars?")

content='🤠🔫💰'


In [15]:
invoke_chain("What movie did I ask about first?")

content='You asked about "Star Wars: Episode IV."'
